# Evaluating Tabular Classifications

## Introduction

In this notebook, we'll walk-through a detailed example of how you can use Valor to evaluate classifications made on a tabular dataset. This example uses `sklearn`'s breast cancer dataset to make a binary prediction about whether a woman has breast cancer, based on a table of descriptive features, such as mean radius and mean texture.

For a conceptual introduction to Valor, [check out our project overview](https://striveworks.github.io/valor/). For a higher-level example notebook, [check out our "Getting Started" notebook](https://github.com/Striveworks/valor/blob/main/examples/getting_started.ipynb).

## Defining Our Datasets

We start by fetching our dataset, dividing it into test/train splits, and uploading both sets to Valor.

In [1]:
from tqdm import tqdm

from sklearn.model_selection import train_test_split
from sklearn.datasets import load_breast_cancer
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.metrics import classification_report

from valor_lite.classification import DataLoader, Classification, MetricType

In [2]:
# load data from sklearn
dset = load_breast_cancer()
dset.feature_names

array(['mean radius', 'mean texture', 'mean perimeter', 'mean area',
       'mean smoothness', 'mean compactness', 'mean concavity',
       'mean concave points', 'mean symmetry', 'mean fractal dimension',
       'radius error', 'texture error', 'perimeter error', 'area error',
       'smoothness error', 'compactness error', 'concavity error',
       'concave points error', 'symmetry error',
       'fractal dimension error', 'worst radius', 'worst texture',
       'worst perimeter', 'worst area', 'worst smoothness',
       'worst compactness', 'worst concavity', 'worst concave points',
       'worst symmetry', 'worst fractal dimension'], dtype='<U23')

In [3]:
# split datasets
X, y, target_names = dset["data"], dset["target"], dset["target_names"]
X_train, X_test, y_train, y_test = train_test_split(X, y)

# show an example input
X_train.shape, y_train[:4], target_names

((426, 30), array([1, 1, 0, 1]), array(['malignant', 'benign'], dtype='<U9'))

### Create GroundTruths

In [4]:
# format training groundtruths
training_groundtruths = [
    (f"train{i}", ("class", target_names[t]))
    for i, t in enumerate(y_train)
]

# format testing groundtruths
testing_groundtruths = [
    (f"test{i}", ("class", target_names[t]))
    for i, t in enumerate(y_test)
]

## Defining Our Model

Now that our `Datasets` have been defined, we can describe our model in Valor using the `Model` object.

In [5]:
# fit an sklearn model to our data
pipe = make_pipeline(StandardScaler(), LogisticRegression())
pipe.fit(X_train, y_train)

# get predictions on both of our datasets
y_train_probs = pipe.predict_proba(X_train)
y_test_probs = pipe.predict_proba(X_test)

# show an example output
y_train_probs[:4]

array([[0.03939074, 0.96060926],
       [0.00546861, 0.99453139],
       [0.88274567, 0.11725433],
       [0.079975  , 0.920025  ]])

### Create Predictions

In [6]:

# define our predictions
training_predictions = [
    (
        f"train{i}",
        [
            (("class", target_names[j]), p)
            for j, p in enumerate(prob)
        ],
    )
    for i, prob in enumerate(y_train_probs)
]
testing_predictions = [
    (
        f"test{i}",
        [
            (("class", target_names[j]), p)
            for j, p in enumerate(prob)
        ],
    )
    for i, prob in enumerate(y_test_probs)
]

## Create Classifications

In [7]:
training_clfs = [
    Classification(
        uid=gt[0],
        groundtruths=[gt[1]],
        predictions=[
            plabel
            for plabel, _ in pd[1]
        ],
        scores=[
            score
            for _, score in pd[1]
        ]
    )
    for gt, pd in zip(training_groundtruths, training_predictions)
    if gt[0] == pd[0]
]

testing_clfs = [
    Classification(
        uid=gt[0],
        groundtruths=[gt[1]],
        predictions=[
            plabel
            for plabel, _ in pd[1]
        ],
        scores=[
            score
            for _, score in pd[1]
        ]
    )
    for gt, pd in zip(testing_groundtruths, testing_predictions)
    if gt[0] == pd[0]
]

## Evaluating Performance

With our `Dataset` and `Model` defined, we're ready to evaluate our performance and display the results. Note that we use the `wait_for_completion` method since all evaluations run as background tasks; this method ensures that the evaluation finishes before we display the results.

In [8]:
training_loader = DataLoader()
training_loader.add_data(training_clfs)
training_evaluator = training_loader.finalize()

testing_loader = DataLoader()
testing_loader.add_data(testing_clfs)
testing_evaluator = testing_loader.finalize()

In [9]:
metrics = testing_evaluator.evaluate()

As a brief sanity check, we can check Valor's outputs against `sklearn's` own classification report. We see that the two results are equal.

In [10]:
metrics[MetricType.Precision]

[Precision(value=[0.0], score_thresholds=[0.5], label=('class', 'malignant')),
 Precision(value=[0.0], score_thresholds=[0.5], label=('class', 'benign'))]

In [11]:
metrics[MetricType.Recall]

[Recall(value=[1.0], score_thresholds=[0.5], label=('class', 'malignant')),
 Recall(value=[0.9789473684210527], score_thresholds=[0.5], label=('class', 'benign'))]

In [12]:
metrics[MetricType.F1]

[F1(value=[0.0], score_thresholds=[0.5], label=('class', 'malignant')),
 F1(value=[0.0], score_thresholds=[0.5], label=('class', 'benign'))]

In [13]:
y_train_preds = pipe.predict(X_train)
print(classification_report(y_train, y_train_preds, digits=6, target_names=target_names))

              precision    recall  f1-score   support

   malignant   0.993711  0.975309  0.984424       162
      benign   0.985019  0.996212  0.990584       264

    accuracy                       0.988263       426
   macro avg   0.989365  0.985760  0.987504       426
weighted avg   0.988324  0.988263  0.988241       426

